In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
nltk.download('wordnet')
import matplotlib.pyplot as plt
from gensim.models.coherencemodel import CoherenceModel
import ast 
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,Dense,Conv1D,Flatten,LSTM,GlobalMaxPooling1D,Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


In [2]:
data = pd.read_csv("clean_data.csv")
text = data['tokens']

data['words'] = data['tokens']

for i in range(len(text)):
    data['words'][i] = ast.literal_eval(data['tokens'][i]) 
    
data['words'] = data['words'].apply(lambda x: " ".join(x) )
text = data['words']

result = []
for i in range(len(text)):
    result.append(text[i])
    
#dictionary = gensim.corpora.Dictionary(result)
#bow_corpus = [dictionary.doc2bow(doc) for doc in result]

vectorizer = CountVectorizer()
result1 = vectorizer.fit_transform(data['words'])

In [8]:
result1

<18177x110111 sparse matrix of type '<class 'numpy.int64'>'
	with 2151812 stored elements in Compressed Sparse Row format>

In [10]:
### TruncatedSVD(LSA)


#from sklearn.decomposition import SparsePCA
#pca = SparsePCA(n_components=2)
#pca.fit(result1.toarray())
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(result1)

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=7, random_state=42,
             tol=0.0)

### Word2Vec + PCA

In [6]:
tokenizer = Tokenizer(num_words = 4000)
tokenizer.fit_on_texts(text)
sequence = tokenizer.texts_to_sequences(text)

In [7]:
index_of_words = tokenizer.word_index
print(len(index_of_words))

110194


In [9]:
padded_seq = pad_sequences(sequence , maxlen = 1000 )

In [10]:
padded_seq

array([[   0,    0,    0, ..., 1090, 1858,   12],
       [   0,    0,    0, ..., 3828, 2085, 1204],
       [   0,    0,    0, ...,  401,  394,   40],
       ...,
       [   0,    0,    0, ..., 2766,  435, 1328],
       [   0,    0,    0, ..., 2328,  654,    4],
       [   0,    0,    0, ...,   13, 3648,  129]])

In [13]:
f = open('glove.6B.300d.txt','rb')
embedd_index = {}
for line in f:
    val = line.split()
    word = val[0]
    coff = np.asarray(val[1:],dtype = 'float')
    embedd_index[word] = coff

f.close()
print('Found %s word vectors.' % len(embedd_index))

Found 400000 word vectors.


In [14]:
embedding_matrix = np.zeros((len(index_of_words) + 1, 300))

tokens = []
labels = []

for word,i in index_of_words.items():
    temp = embedd_index.get(word)
    if temp is not None:
        embedding_matrix[i] = temp
        
#for plotting
        tokens.append(embedding_matrix[i])
        labels.append(word)

In [15]:
embedding_matrix.shape

(110195, 300)

In [16]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(embedding_matrix)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\decomposition\pca.py:536: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ / total_var.sum()


PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)